<a href="https://colab.research.google.com/github/Megaptera666/Linearized-electrodiffusion/blob/master/Linearized_electrodiffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Electrodiffusion of K<sup>+</sup> through small, tortuous extracellular spaces in the presence of K<sup>+</sup> sources and sinks
##Application of a linearized version of the Nernst-Plank equation


#Problem definition

<p>The brain is comprised of millions of neurons that incenssantly produce cognitive computations. Unlike in the cables and chips of man-made circuits, the electric signals underlying these computations are mediated by the flux of of ions such as K<sup>+</sup>. The number of ions required to carry electric currents is minute. However, in some cases, neurons are so densely packed that the extracellular concentration of K<sup>+</sup> ([K]<sub>o</sub>, in mol cm<sup>-3</sup>) is prone to change, making the regulation of [K]<sub>o</sub> a widespread challenge throughout the brain.

<p>This script calculates [K]<sub>o</sub> as a function of position in a two-dimentional space in the presence of K<sup>+</sup> sources and sinks and in the presence of an extracellular electric field. The script utilizes the numerical method of lines to calculate the rate of change in [K]<sub>o</sub>(x) (in mol cm <sup>-3</sup> s<sup>-1</sup>), as follows.

<p>The [continuity equation](https://en.wikipedia.org/wiki/Continuity_equation) dictates that, if the lateral flux of K<sup>+</sup> (J<sup>K</sup><sub>electrodiffusion</sub> (x) , in mol cm<sup>-2</sup> s<sup>-1</sup>) is not in balance with the sources or sinks of K<sup>+</sup> (Q<sub>s</sub>, in mol cm<sup>-3</sup> s<sup>-1</sup>), then [K]<sub>o</sub> (x) must be changing in time:
$
\begin{align}
\frac{\partial [K]_o(x)}{\partial t} &= -\frac{\partial J^K_{electrodiffusion}(x)}{\partial x} + Q_s(x) \tag{1}
\end{align}$

The [Nernst-plank equation](https://en.wikipedia.org/wiki/Nernst%E2%80%93Planck_equation) describes the flux of ions under the influence of both an ionic concentration gradient ($\frac{\partial [K]_o}{\partial x}$ in mol cm <sup>-2</sup>) and an electric field ($\frac{\partial Vo}{\partial x}$, in V cm <sup>-1</sup>)

$
\begin{align}
 J^K_{electrodiffusion}(x) =- D^* \bigg[\frac{\partial [K]_o}{\partial x} + r [K]_o \frac{\partial Vo}{\partial x} \bigg] \tag{2}
 \end{align}$

#Problem definition 

In [3]:
#@title Libraries {display-mode: "form"}
import numpy as np
import scipy.integrate as integrate
from numpy.core._multiarray_umath import ndarray
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import rc #This is to display the animation

In [4]:
#@title Geometry and time {display-mode: "form"}
#@markdown Total length [um]:
length_x = 30  #@param{type:"number"} 
#@markdown delta x [um]:
delta_x = 0.1 #@param {type:"number"}
x_steps = round(length_x / delta_x)

#@markdown total time [ms]
length_t = 200  #@param {type:"number"}
#@markdown delta t [ms]
delta_t = 1.  #@param {type:"number"}
t_steps = round(length_t / delta_t)


In [6]:
#@title Physical properties {display-mode: "form"}
#@markdown Tortuosity factor
lambd = 1. #@param {type:"number"}
#@markdown Difussion coefficient of K in ringers [um^2 s-1]
Diffusion_coefficient = 2000  #@param {type:"number"} 

#@markdown sources and sinks 
#@markdown Magnitude [mM ms-1]
source1 = 5  #@param {type:"number"} 
source2 =-4  #@param {type:"number"} 

#@markdown location [um]
source1_start = 0  #@param {type:"number"} 
source1_end = 20  #@param {type:"number"}
source2_start =20  #@param {type:"number"} 
source2_end = 30  #@param {type:"number"} 



#Problem instantiation

In [ ]:
#@title Initial conditions
Initial_ko_x = np.zeros(x_steps)
Vo_x = np.zeros(x_steps)
SRS_alpha = np.zeros(x_steps)
SRS_tortuosity = np.zeros(x_steps)
D_x = np.zeros(x_steps)
source_x = np.zeros(x_steps)
sink_x = np.zeros(x_steps)
x_dim = np.linspace(0, length_x, x_steps)

# Parametrize initial concentration
Initial_ko_x[:] = 1.#x_dim[:] / 100.  # [mM]


# Parametrize SRS alpha
SRS_alpha.fill(1)

# Parametrize tortuosity
SRS_tortuosity.fill(lambd)

# Calculate effective diffusion coefficient
D_x[:] = Diffusion_coefficient / SRS_tortuosity[:] ** 2

# Parametrize Vo_x
Vo_x[:] = 1.  

# Calculate dVodx
dVodx = np.gradient(Vo_x, delta_x)

# Parametrize sources and sinks
source_x[0:round(10 / delta_x)] = 5.  # [mM ms-1]
# print(source_x)
sink_x[round(len(sink_x) - 10 / delta_x):] = -4.  # [mM ms-1]